# Image Classification Using Convolutional Neural Networks

Code adapted from 'Maratyszcza' (GitHub)

https://github.com/pytorch/examples/blob/42e5b996718797e45c46a25c55b031e6768f8440/imagenet/main.py

In [ ]:
import argparse
import os
import shutil
import time

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import easydict

#### Different pre-trained models to choose from

In [ ]:
model_names = sorted(name for name in models.__dict__
    if name.islower() and not name.startswith("__")
    and callable(models.__dict__[name]))
print(model_names)

#### Defining some functions

In [ ]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #target = target.cuda(async=True) #move target tensor to GPU
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses, top1=top1, top5=top5))


In [ ]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target) in enumerate(val_loader):
        #target = target.cuda(async=True) #move target tensor to GPU
        input_var = torch.autograd.Variable(input, volatile=True)
        target_var = torch.autograd.Variable(target, volatile=True)

        # compute output
        output = model(input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data[0], input.size(0))
        top1.update(prec1[0], input.size(0))
        top5.update(prec5[0], input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))

    print(' * Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))

    return top1.avg

In [ ]:
def save_checkpoint(state, is_best, root_dir, filename='checkpoint.pth.tar'):
    torch.save(state, root_dir + filename)
    if is_best:
        shutil.copyfile(root_dir + filename, root_dir + 'model_best.pth.tar')
    print('Checkpoint Saved.')

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = args.lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

#### Main definition

In [ ]:
best_prec1 = 0

def main():
    global args, best_prec1
    args = easydict.EasyDict({
        "data": '', #path to folder with images
        "arch": 'densenet121', #chosen pre-trained model
        "workers": 0,
        "epochs": 10,
        "start_epoch": 0,
        "batch_size": 4,
        "lr": 0.001,
        "momentum": 0.9,
        "weight_decay": 1e-4,
        "print_freq": 10,
        "resume": '', #path to checkpoint file
        "model_best": '', #path to model_best file
        "evaluate": True,
        "pretrained": True,
        "cuda": False, #set True if training on GPU
        "check_dir": '' #path to checkpoint folder
    })
    
    print("=> training with the following parameters:")
    print(args)

    # create model
    if args.pretrained:
        print("=> using pre-trained model '{}'".format(args.arch))
        model = models.__dict__[args.arch](pretrained=True)
    else:
        print("=> creating model '{}'".format(args.arch))
        model = models.__dict__[args.arch]()
    #set up GPU (if applicable) 
    if args.cuda: ##I ADDED THIS
        if args.arch.startswith('alexnet') or args.arch.startswith('vgg'):
            model.features = torch.nn.DataParallel(model.features)
            model.cuda()
        else:
            model = torch.nn.DataParallel(model).cuda()

    # optionally resume from a checkpoint
    if args.model_best:
        if os.path.isfile(args.model_best):
            print("=> loading best model '{}'".format(args.model_best))
            best = torch.load(args.model_best)
            args.start_epoch = best['epoch']
            best_prec1 = best['best_prec1']
            model.load_state_dict(best['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.model_best, best['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.model_best))
                  
    elif args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            best_prec1 = checkpoint['best_prec1']
            model.load_state_dict(checkpoint['state_dict'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    cudnn.benchmark = True

    # Data loading code
    traindir = os.path.join(args.data, 'train')
    valdir = os.path.join(args.data, 'val')
    normalize = transforms.Normalize(mean = [0.20572032, 0.28209996, 0.29277292], 
                                     std = [0.065873735, 0.08655865, 0.1461992]) #Compute mean and std with ImageStats.ipynb

    train_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(traindir, transforms.Compose([
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=args.batch_size, shuffle=True,
        num_workers=args.workers, pin_memory=True)

    val_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            normalize,
        ])),
        batch_size=args.batch_size, shuffle=False,
        num_workers=args.workers, pin_memory=True)
    
    print("=> initialized data loaders and transformations")
    # define loss function (criterion) and pptimizer
    if args.cuda:
        criterion = nn.CrossEntropyLoss().cuda()
    else: 
        criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.SGD(model.parameters(), args.lr,
                                momentum=args.momentum,
                                weight_decay=args.weight_decay)
    print("=> defined optimizer function")
    if args.evaluate:
        validate(val_loader, model, criterion)
        return

    for epoch in range(args.start_epoch, args.epochs):
        adjust_learning_rate(optimizer, epoch)

        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch)

        # evaluate on validation set
        prec1 = validate(val_loader, model, criterion)

        # remember best prec@1 and save checkpoint
        is_best = prec1 > best_prec1
        best_prec1 = max(prec1, best_prec1)
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': args.arch,
            'state_dict': model.state_dict(),
            'best_prec1': best_prec1,
        }, is_best, args.check_dir)

In [ ]:
if __name__ == '__main__':
    main()

# Making the Prediction

In [ ]:
import pickle
def load_lbl_lst(filename, image_dir):
    objects = dict()
    with open(filename, 'rb') as f:
        try:
            while True:
                objects.update(pickle.load(f))
        except Exception:
            pass
    return objects[image_dir]

In [ ]:
import time
def save_pickle():
    dictio = {image_dir: lbl_lst, "reviewed_nr":i+1}
    today = time.strftime("%Y%m%d-%H%M%S")
    filename = '' #filename for label progress file
    pickle.dump(dictio, open(filename+today+'.p', 'wb'))

In [ ]:
import csv
def load_grid_data(filename):
    grid_data = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for line in reader:
            if line[0]=='id':
                header = line
            else:
                grid_data.append(line)
    return grid_data, header

In [ ]:
# Loading the best model
model_best = '' #path to model_best file
arch = 'densenet121' #chosen pre-trained model

print("=> using pre-trained model '{}'".format(arch))
model = models.__dict__[arch](pretrained=True)
    
if os.path.isfile(model_best):
    print("=> loading best model '{}'".format(model_best))
    best = torch.load(model_best)
    best_prec1 = best['best_prec1']
    model.load_state_dict(best['state_dict'])
    print("=> loaded checkpoint '{}' (best prec1 {})"
          .format(model_best, best['best_prec1']))
else:
    print("=> no checkpoint found at '{}'".format(model_best))

testdir = '' #path to images to label
test_loader = torch.utils.data.DataLoader(
        datasets.ImageFolder(testdir, transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize(mean = [0.20572032, 0.28209996, 0.29277292], 
                                std = [0.065873735, 0.08655865, 0.1461992]),
        ])),
        batch_size=1, shuffle=False,
        num_workers=0, pin_memory=True)

# Loading label list
file = '' #path to progress file (pickle)
lbl_lst = load_lbl_lst(file, testdir)
total = len(lbl_lst)

# Running predictions
model.eval()
labels = []
for i, data in enumerate(test_loader):
    if i%100==99:
        print("Progress: {} %".format(round(i/total*100,2)))
        save_pickle()
    if lbl_lst[i]==-1:
        print('labeling...')
        input = data[0]
        input_var = torch.autograd.Variable(input, volatile=True)
        output = model(input_var)
        _, predicted = torch.max(output.data, 1)
        lbl_lst[i] = predicted[0].item()
save_pickle()
    